In [22]:
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from stem import Signal
from stem.control import Controller
import time
from datetime import datetime

# Function to request new Tor identity (new IP address)
def renew_tor_identity():
    with Controller.from_port(port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM) 

# Configure Selenium to use Tor
def get_tor_firefox_browser():
    options = Options()
    options.headless = True  # Run in headless mode (no GUI)
    options.set_preference('network.proxy.type', 1)
    options.set_preference('network.proxy.socks', '127.0.0.1')
    options.set_preference('network.proxy.socks_port', 9050)
    options.set_preference('network.proxy.socks_remote_dns', True)  
    browser = webdriver.Firefox(options=options)
    return browser

In [3]:
# Initialize Firefox with Tor
#browser = setup_tor_firefox()
browser = get_tor_firefox_browser()

# Visit the website
url = 'http://mbrlkbtq5jonaqkurjwmxftytyn2ethqvbxfu4rgjbkkknndqwae6byd.onion/index.php?page=1'  # Replace with your target URL
browser.get(url)
time.sleep(5)  # Allow time for page to load

In [34]:
# Scrape data (adjust as per the website structure)
#elements = browser.find_elements_by_xpath('//h1')  # Change based on what you need to scrape
companyNames = []
locations = []
dateVictimised = []
noClicks = []
publishedStatus = []
linksToData = []
websites = []

# Visit the website
for i in range(21):
    browser.get(f'http://mbrlkbtq5jonaqkurjwmxftytyn2ethqvbxfu4rgjbkkknndqwae6byd.onion/index.php?page={i+1}')
    time.sleep(5)  # Allow time for page to load
    
    news_elements = browser.find_elements(By.XPATH,"//tbody/tr/th[@class='News']")
        
    for news_element in news_elements:
        # Extract the 'onClick' attribute and the visible text for each element
        onclick_value = news_element.get_attribute('onclick')
        onclick_value = onclick_value[11:len(onclick_value)-2]
        text_content = news_element.text
        
        # Split the text content by newlines to separate the text into different parts
        text_parts = text_content.split('\n')
        
        company_name = text_parts[0]
        location = text_parts[1]
        formatted_websites = text_parts[2].lstrip().split()
        for formatted_website in formatted_websites:
            website = " | ".join(formatted_websites)
         
        views = text_parts[3].split(': ')[1]
        publication_date = datetime.strptime(text_parts[5].split(': ')[1], '%Y-%m-%d')
        formatted_date = publication_date.strftime('%d/%m/%Y')      # Format the date into DD/MM/YYY
        status = text_parts[6]
        print(f"Scraping: {company_name}")
        
        companyNames.append(company_name)
        locations.append(location)
        websites.append(website)
        dateVictimised.append(formatted_date)
        noClicks.append(views)
        linksToData.append(f"http://mbrlkbtq5jonaqkurjwmxftytyn2ethqvbxfu4rgjbkkknndqwae6byd.onion/topic.php?id={onclick_value}")
        if "PUBLISHED" in status:
            publishedStatus.append("Published")
        else:
            publishedStatus.append("Not Published")

    
#browser.quit()

# Store data
#data = [{'Title': element.text} for element in elements]

# Close the browser
#browser.quit()

# Save data into CSV
print("Scraping completed")

Scraping: Garvey
Scraping: Classic Business Products
Scraping: The Rubber Resources
Scraping: Condere Ip, Infracom Group
Scraping: G/S Solutions
Scraping: Andantex USA
Scraping: 4B Components
Scraping: Reutter
Scraping: Performance Food Centers
Scraping: InteriorWorx Commercial Flooring
Scraping: Affirm Agency
Scraping: Mile Hi Foods
Scraping: Eurobulk
Scraping: BroadGrain Commodities
Scraping: Thompson Construction Supply
Scraping: Pacific Coast Building Products
Scraping: Omega Industries
Scraping: Noble Environmental
Scraping: Messe C
Scraping: Jackson Paper Manufacturing
Scraping: Plaisted Companies
Scraping: Multidata
Scraping: Hariri Pontarini Architects
Scraping: Rsp
Scraping: Protective Industrial Products
Scraping: Baskervill
Scraping: Creative Playthings
Scraping: Dimensional Merchandising
Scraping: True Family Enterprises
Scraping: Southeast Cooler
Scraping: Virginia Dare Extract Co.
Scraping: Joe Swartz Electric
Scraping: Bel-Air Bay Club
Scraping: Elgin Separation Solution

In [35]:
print("Company Names:", companyNames)
print("Date Victimised:", dateVictimised)
print("Number of Clicks:", noClicks)
print("Published Status:", publishedStatus)
print("Websites:", websites)
print("LinksToData:", linksToData)

Company Names: ['Garvey', 'Classic Business Products', 'The Rubber Resources', 'Condere Ip, Infracom Group', 'G/S Solutions', 'Andantex USA', '4B Components', 'Reutter', 'Performance Food Centers', 'InteriorWorx Commercial Flooring', 'Affirm Agency', 'Mile Hi Foods', 'Eurobulk', 'BroadGrain Commodities', 'Thompson Construction Supply', 'Pacific Coast Building Products', 'Omega Industries', 'Noble Environmental', 'Messe C', 'Jackson Paper Manufacturing', 'Plaisted Companies', 'Multidata', 'Hariri Pontarini Architects', 'Rsp', 'Protective Industrial Products', 'Baskervill', 'Creative Playthings', 'Dimensional Merchandising', 'True Family Enterprises', 'Southeast Cooler', 'Virginia Dare Extract Co.', 'Joe Swartz Electric', 'Bel-Air Bay Club', 'Elgin Separation Solutions', 'PIGGLY WIGGLY ALABAMA DISTRIBUTING', 'Weldco-Beales Manufacturing', 'Evans Distribution Systems', 'Seirus Innovation', 'Crain Group', 'Bakersfield', "Farmers' Rice Cooperative", 'Clabots', 'Omicron Granite & Tile', 'Uni

In [39]:
dataSize = []
dataType = []
companyDescription = []


for link in linksToData:
    browser.get(link)
    time.sleep(5)

    try:
        news_element = browser.find_element(By.XPATH, "//tbody/tr/th[@class='News']")
        text_content = news_element.text
        
        # Split the text content into lines
        lines = text_content.splitlines()

        # Extract the amount of data, company info, and comments
        # Assuming the lines follow a consistent order
        for line in lines:
            if "amount of data:" in line:
                size = line.split("amount of data:")[1].strip()
                dataSize.append(size)

            elif "information:" in line:
                info = line.split("information:")[1].strip()
                companyDescription.append(info)

            elif "comment:" in line:
                comment = line.split("comment:")[1].strip()
                dataType.append(comment)

    except Exception as e:
        print(f"Error extracting data from {link}: {e}")

print("Data Sizes:", dataSize)
print("Company Descriptions:", companyDescription)
print("Comments:", dataType)


print("Scraping completed")

Data Sizes: ['??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '??? gb', '596 gb', '??? gb', '28 gb', '43 gb', '37 gb', '322 gb', '11 gb', '??? gb', '29 gb', '61 gb', '146 gb', '103 gb', '33 gb', '149 gb', '159 gb', '36 gb', '63 gb', '113 gb', '176 gb', '129 gb', '428 gb', '593 gb', '19 gb', '18 gb', '185 gb', '28 gb', '??? gb', '31 gb', '345 gb', '42 gb', '??? gb', '397 gb', '26 gb', '17 gb', '164 gb', '42 gb', '167 gb', '??? gb', '228 gb', '288 gb', '205 gb', '31 gb', '69 gb', '69 gb', '169 gb', '12 gb', '15 gb', '??? gb', '107 gb', '126 gb', '113 gb', '??? gb', '160 gb', '68 gb', '68 gb', '278 gb', '839 gb', '54 gb', '280 gb', '27 gb', '76 gb', '139 gb', '216 gb', '87 gb', '??? gb', '70 gb', '417 gb', '56 gb', '40 gb', '131 gb', '35 gb', '296 gb', '549 gb', '10 gb', '839 gb', '632 gb', '168 gb', '99 gb', '15 gb', '3

In [52]:
cleanedDataSize = []

for formatDataSize in dataSize:
    var = formatDataSize.split()
    if "???" in var:
        cleanedDataSize.append("NA")
    else:
        cleanedDataSize.append(var[0])

df = pd.DataFrame({"Company Name": companyNames, "Company Website" : websites, "Location" : locations, "Company Description": companyDescription, "Date Victimised": dateVictimised, "Number of Clicks": noClicks, "Status": publishedStatus, "Data Size (Gb)": cleanedDataSize, "Data Type" : dataType})
df['Date Victimised'] = pd.to_datetime(df['Date Victimised'], dayfirst=True)
date_mask_month = df['Date Victimised'].dt.month > 8
date_mask_yr = df['Date Victimised'].dt.year < 2024
df = df[~date_mask]

df = df.reset_index(drop=True)

df.to_csv('play.csv', index=False)

print("Scraping completed and saved to scraped play.csv")

Scraping completed and saved to scraped play.csv


In [53]:
import difflib
import pandas as pd

# Load the CSV file into a dataframe
file_path = 'play.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Define separate keyword lists for each column
keywords_dict = {
    "Financial Data": ["finance", "financial", "budget", "accounting"],
    "PII": ["personal", "identifiable", "pii"],
    "Customer and Client Data": ["customer", "client", "consumer"],
    "Employee and HR data": ["employee", "hr", "human resources", "hr data"],
    "Legal Documents": ["legal", "court", "casefile", "litigation"],
    "Intellectual Property and Trade Secrets": ["intellectual property", "trade secret", "ip", "design"],
    "Databases": ["database", "databases", "db"],
    "Sensitive Communication": ["communication", "email", "chat", "message", "correspondence"],
    "Healthcare Data": ["health", "medical", "healthcare"],
    "Utility and Infrastructure Data": ["utility", "infrastructure", "power", "water"]
}

# Initialize the new columns in the dataframe with False values
for col in keywords_dict:
    df[col] = False
    
df['Data Type'] = df['Data Type'].str.replace('\n', ' ', regex=False)


# Function to find the closest match for each category
def match_keywords(data_type, keywords_dict):
    matches = {col: False for col in keywords_dict}
    for col, keywords in keywords_dict.items():
        for keyword in keywords:
            if difflib.get_close_matches(data_type, keywords, cutoff=0.7):
                matches[col] = True
    return matches

# Apply the matching logic to the dataframe
def update_sensitive_columns_difflib(row):
    data_type = str(row['Data Type']).lower() if pd.notna(row['Data Type']) else ""
    
    # Split the 'Data Type' column by whitespace (in case entries are separated by newline or spaces)
    data_entries = data_type.split(' ')
    
    # Initialize column match flags
    column_matches = {col: False for col in keywords_dict}
    
    # Check each entry in the data type
    for entry in data_entries:
        entry_matches = match_keywords(entry.strip(), keywords_dict)
        
        # Update match flags for each column
        for col, match in entry_matches.items():
            if match:
                column_matches[col] = True
    
    # Update the row with the match results
    for col, match in column_matches.items():
        row[col] = match
    
    return row

# Apply the function to each row of the dataframe
df = df.apply(update_sensitive_columns_difflib, axis=1)

# Display the updated dataframe to the user
df.head()
df.to_csv('play_cleaning.csv', index=False)